# Prepare training data to compute the guess function

In [7]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import pandas as pd
import numpy as np
import random
import itertools
import math
import time
from typing import Optional, Tuple, cast, List, Dict

## prepare all random configurations

In [63]:
partitions = 20
random_configuration_setup = {'state_probabilities': { 'min_value': 0.25, 'max_value': 1, 'partitions': 20},
                            'angles_rx': { 'min_value': 0, 'max_value': 2*np.pi, 'partitions': 20},
                            'angles_ry': { 'min_value': 0, 'max_value': 2*np.pi, 'partitions': 20},
                            'etas': { 'min_value': 0, 'max_value': np.pi/2, 'partitions': 20, 'only_entangled': True}}
number_random_configuration = 50

In [55]:
def prepare_random_configurations(configuration_setup: dict, number_random_configuration: int):
    """ create random configurations from a given range for each variable """
    if ('state_probabilities' not in configuration_setup or 
        'angles_rx' not in configuration_setup or 
        'angles_ry' not in configuration_setup or 
        'etas' not in configuration_setup):
        raise ValueError('configuration_setup must include state_probabilities, angles_rx, angles_ry and etas')
    state_probabilities = get_variable_distribution_values(configuration_setup['state_probabilities']['min_value'],
                                                          configuration_setup['state_probabilities']['max_value'],
                                                          configuration_setup['state_probabilities']['partitions'])
    angles_rx = get_variable_distribution_values(configuration_setup['angles_rx']['min_value'],
                                                          configuration_setup['angles_rx']['max_value'],
                                                          configuration_setup['angles_rx']['partitions'])
    angles_ry = get_variable_distribution_values(configuration_setup['angles_ry']['min_value'],
                                                          configuration_setup['angles_ry']['max_value'],
                                                          configuration_setup['angles_ry']['partitions'])
    eta_pairs = (get_entangled_eta_pairs(configuration_setup['etas']['min_value'],
                                        configuration_setup['etas']['max_value'],
                                        configuration_setup['etas']['partitions']) if configuration_setup['etas']['only_entangled'] is True 
                 else get_eta_pairs(configuration_setup['etas']['min_value'],
                                   configuration_setup['etas']['max_value'],
                                   configuration_setup['etas']['partitions']))
    
    return (state_probabilities, angles_rx, angles_ry, eta_pairs)

In [11]:
def get_variable_distribution_values(min_value=float, max_value=float, partitions=int) -> List[float]:
    """ Return a list of values distributed evenly from min to max and added the last max_value at the end """
    return np.append(np.arange(min_value, max_value, max_value/partitions), max_value)

In [104]:
def get_number_samples_eta_pairs_and_rest(total_configurations:int, n_state_probs:int, n_angles_rx:int, n_angles_ry:int, n_eta_pairs:int) -> Tuple[int, int, int, int, int]:
    """ Return the number of samples for each list, evenly distributed, and adding more eta pairs samples when not possible """
    configs_per_list = math.modf(total_configurations ** (1/4))[1]
    added_configs = total_configurations - configs_per_list ** 4
    n_configs_per_state_probs = min(n_state_probs, configs_per_list)
    n_configs_per_angles_rx = min(n_angles_rx, configs_per_list)
    n_configs_per_angles_ry = min(n_angles_ry, configs_per_list)
    added_configs += (configs_per_list - n_configs_per_state_probs) + (configs_per_list - n_configs_per_angles_rx) + (configs_per_list - n_configs_per_angles_ry)
    n_configs_per_eta_pairs = min(n_eta_pairs, configs_per_list)
    return (n_configs_per_state_probs, n_configs_per_angles_rx, n_configs_per_angles_ry, n_configs_per_eta_pairs, added_configs)

In [108]:
get_number_samples_eta_pairs_and_rest(9999, 20, 20, 20, 20)

(9.0, 9.0, 9.0, 9.0, 3438.0)

In [103]:
3*3*3*4

108

In [59]:
test = [0,1,2,3,4,5,6,7,8,9,10]

In [62]:
random.sample(test, 10)

[0, 10, 4, 1, 5, 2, 9, 7, 6, 8]

In [80]:
max_configs = 2

In [90]:
configs = math.modf(100 ** (1/4))

In [83]:
remaining_configs = configs[1] - max_configs

In [84]:
remaining_configs

1.0

In [97]:
100 - configs[1] ** 4

19.0

In [94]:
(100 ** (1/4))**4

100.00000000000003

In [91]:
configs

(0.16227766016837952, 3.0)

In [89]:
configs[0]*100

16.227766016837954

In [75]:
100-3**4

19

In [76]:
20**4

160000

In [88]:
19 ** (1/4)

2.087797629929844

In [58]:
prepare_random_configurations(configuration_setup=random_configuration_setup)

(array([0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75,
        0.8 , 0.85, 0.9 , 0.95, 1.  ]),
 array([0.        , 0.31415927, 0.62831853, 0.9424778 , 1.25663706,
        1.57079633, 1.88495559, 2.19911486, 2.51327412, 2.82743339,
        3.14159265, 3.45575192, 3.76991118, 4.08407045, 4.39822972,
        4.71238898, 5.02654825, 5.34070751, 5.65486678, 5.96902604,
        6.28318531]),
 array([0.        , 0.31415927, 0.62831853, 0.9424778 , 1.25663706,
        1.57079633, 1.88495559, 2.19911486, 2.51327412, 2.82743339,
        3.14159265, 3.45575192, 3.76991118, 4.08407045, 4.39822972,
        4.71238898, 5.02654825, 5.34070751, 5.65486678, 5.96902604,
        6.28318531]),
 [(1.0995574287564276, 1.0210176124166828),
  (1.1780972450961724, 0.9424777960769379),
  (1.1780972450961724, 1.0210176124166828),
  (1.1780972450961724, 1.0995574287564276),
  (1.2566370614359172, 0.8639379797371931),
  (1.2566370614359172, 0.9424777960769379),
  (1.2566370614359172, 1.021017612

In [15]:
get_variable_distribution_values(0.25,1,20)

array([0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75,
       0.8 , 0.85, 0.9 , 0.95, 1.  ])

In [8]:
def get_combinations_two_etas_without_repeats_from_etas(angles_etas: List[float]) -> List[Tuple[float, float]]:
    """ from a given list of attenuations factors create a
        list of all combinatorial pairs of possible etas
        without repeats
        For us it is the same testing first eta 0.1 and second eta 0.2
        than first eta 0.2 and second eta 0.1
        Though, we will always put the greater value as the first pair element
    """
    # when there is only one element, we add the same element
    if len(angles_etas) == 1:
        angles_etas.append(angles_etas[0])
    # get combinations of two etas without repeats
    eta_pairs = list(itertools.combinations(angles_etas, 2))

    return reorder_pairs(eta_pairs)

def reorder_pairs(pairs: List[Tuple[float, float]]) -> List[Tuple[float, float]]:
    """ reorder received pairs setting first the element of the tuple
        as greater or equal de second one
    """
    reordered_pairs = pairs
    for idx, pair in enumerate(pairs):
        reordered_pairs[idx] = reorder_pair(pair)
    return reordered_pairs

def reorder_pair(pair: Tuple[float, float]) -> Tuple[float, float]:
    if pair[0] < pair[1]:
        return (pair[1], pair[0])
    return pair

In [38]:
def get_eta_pairs(min_value: float, max_value: float, num_etas_to_be_split: int)-> List[Tuple[float, float]]:
    etas = get_variable_distribution_values(min_value, max_value, num_etas_to_be_split)
    return get_combinations_two_etas_without_repeats_from_etas(etas)

In [57]:
def get_entangled_eta_pairs(min_value: float, max_value: float, num_etas_to_be_split: int)-> List[Tuple[float, float]]:
    return get_entangled_ys(min_value=min_value, max_value=max_value, num_etas_to_be_split=num_etas_to_be_split)[3]

In [41]:
def get_entangled_ys(num_etas_to_be_split: int, min_value: float = 0, max_value: float=np.pi/2 )-> Tuple[List[float], Tuple[int, int, int]]:
    eta_pairs = get_eta_pairs(min_value, max_value, num_etas_to_be_split)
    gammas = [np.cos(eta_pair[1]) + np.cos(eta_pair[0]) for eta_pair in eta_pairs]
    gammas_etas = [(eta_pair[0], eta_pair[1], gammas[idx]) for idx, eta_pair in enumerate(eta_pairs)]
    entangled_gammas = [gamma for gamma in gammas if gamma <= 1]
    entangled_eta_pairs = [(eta_pair[0], eta_pair[1]) for idx, eta_pair in enumerate(eta_pairs) if gammas[idx] <= 1]
    entangled_eta_pairs.sort()
    theoretical_ys = [(gamma-1)/(gamma-2) for gamma in gammas] 
    entangled_ys = [theoretical_y for theoretical_y in theoretical_ys if theoretical_y > 0]
    entangled_ys.sort()
    entangled_ys.reverse()
    return (entangled_ys, (len(entangled_ys), max(entangled_ys), min(entangled_ys)), entangled_gammas, entangled_eta_pairs, eta_pairs)

In [17]:
def get_gammas_min_max_from_entangled_ys(ty_min:float = 0, ty_max:float = 0.5):
    gamma_min = (1-2*ty_min)/(1-ty_min)
    gamma_max = (1-2*ty_max)/(1-ty_max)
    return (gamma_min, gamma_max)

In [42]:
len(get_entangled_ys(20)[3])

60

In [43]:
get_entangled_ys(20)[3]

[(1.0995574287564276, 1.0210176124166828),
 (1.1780972450961724, 0.9424777960769379),
 (1.1780972450961724, 1.0210176124166828),
 (1.1780972450961724, 1.0995574287564276),
 (1.2566370614359172, 0.8639379797371931),
 (1.2566370614359172, 0.9424777960769379),
 (1.2566370614359172, 1.0210176124166828),
 (1.2566370614359172, 1.0995574287564276),
 (1.2566370614359172, 1.1780972450961724),
 (1.335176877775662, 0.7068583470577035),
 (1.335176877775662, 0.7853981633974483),
 (1.335176877775662, 0.8639379797371931),
 (1.335176877775662, 0.9424777960769379),
 (1.335176877775662, 1.0210176124166828),
 (1.335176877775662, 1.0995574287564276),
 (1.335176877775662, 1.1780972450961724),
 (1.335176877775662, 1.2566370614359172),
 (1.413716694115407, 0.6283185307179586),
 (1.413716694115407, 0.7068583470577035),
 (1.413716694115407, 0.7853981633974483),
 (1.413716694115407, 0.8639379797371931),
 (1.413716694115407, 0.9424777960769379),
 (1.413716694115407, 1.0210176124166828),
 (1.413716694115407, 1.09

## run all configurations (without guess) on a parametrized circuit

## convert execution result counts and configuration to a training row data

In [2]:
one_execution = { 'state_probability': 0, # [0.25, 1]
              'rx_theta': 0, # [0, 2*pi]
              'ry_theta': 0, # [0, 2*pi]
              'eta0': 0, # [1.0995574287564276, pi/2] -> only computed for 20 etas, and it should be the resulting entangled etas for eta0 [63º, 90º]
              'eta1': 0, # [0, pi/2] -> but this is a discrete variable only 20 values
              'bit0': 0, # [0 or 1] the left bit of the measured counts '00'
              'bit1': 0, # [0 or 1] the right bit of the measured counts '00'
              'eta_used': 0} # target value

state_probabilities = []
rx_thetas = []
ry_thetas = []
etas0 = []
etas1 = []
bits0 = []
bits1 = []
etas_used = []
data = { 'state_probability': state_probabilities,
              'rx_theta': rx_thetas,
              'ry_theta': ry_thetas,
              'eta0': etas0,
              'eta1': etas1,
              'bit0': bits0,
              'bit1': bits1,
              'eta_used': etas_used} 

## write training data to a CSV file

In [5]:
df = pd.DataFrame(data)
csv_file_name = "./entangled_executions.csv"
df.to_csv(csv_file_name)